# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[HSE][ML][MS][HW09] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [2]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import preprocessing
import pandas
import random
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier
from typing import Callable, Union, NoReturn, Optional, Dict, Any, List

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [3]:
def gini(x: np.ndarray):
    """
    Считает коэффициент Джини для массива меток x.
    """
    n = len(x)
    _, probs = np.unique(x, return_counts=True)
    probs = probs/n
    return np.sum(probs*(1-probs))

def entropy(x: np.ndarray):
    """
    Считает энтропию для массива меток x.
    """
    n = len(x)
    _, probs = np.unique(x, return_counts=True)
    probs = probs/n
    return -np.sum(probs*np.log2(probs))

def gain(left_y: np.ndarray, right_y: np.ndarray, criterion: Callable):
    """
    Считает информативность разбиения массива меток.

    Parameters
    ----------
    left_y : np.ndarray
        Левая часть разбиения.
    right_y : np.ndarray
        Правая часть разбиения.
    criterion : Callable
        Критерий разбиения.
    """
    i_total = criterion(np.hstack((left_y, right_y)))
    i_left = criterion(left_y)
    i_right = criterion(right_y)

    ig = i_total - (i_left*len(left_y)+ i_right*len(right_y))
    return ig

In [4]:
def bagging(X, y):
    n = X.shape[0]
    num = np.arange(n)
    mask_bag = np.random.choice(num, size=n)
    mask_oob = ~np.in1d(num, mask_bag)
    return (X[mask_bag], y[mask_bag]), (X[mask_oob], y[mask_oob])


In [5]:
class DecisionTreeLeaf:
    """

    Attributes
    ----------
    y : Тип метки (напр., int или str)
        Метка класса, который встречается чаще всего среди элементов листа дерева
    prob: dict
        словарь вероятностей меток
    """
    def __init__(self, y_leaf):
        classes, freq = np.unique(y_leaf, return_counts=True)
        self.y = classes[np.argmax(freq)]
        self.prob = {classes[i]: freq[i]/len(y_leaf) for i in range(len(classes))}


class DecisionTreeNode:
    """

    Attributes
    ----------
    split_dim : int
        Измерение, по которому разбиваем выборку.
    split_value : float
        Значение, по которому разбираем выборку.
    left : Union[DecisionTreeNode, DecisionTreeLeaf]
        Поддерево, отвечающее за случай x[split_dim] < split_value.
    right : Union[DecisionTreeNode, DecisionTreeLeaf]
        Поддерево, отвечающее за случай x[split_dim] >= split_value.
    """
    def __init__(self, split_dim: int, split_value: float,
                 left: Union['DecisionTreeNode', DecisionTreeLeaf],
                 right: Union['DecisionTreeNode', DecisionTreeLeaf]):
        self.split_dim = split_dim
        self.split_value = split_value
        self.left = left
        self.right = right

In [6]:
class DecisionTreeClassifier:
    """
    Attributes
    ----------
    root : Union[DecisionTreeNode, DecisionTreeLeaf]
        Корень дерева.

    (можете добавлять в класс другие аттрибуты).

    """
    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):
        """
        Parameters
        ----------
        criterion : str
            Задает критерий, который будет использоваться при построении дерева.
            Возможные значения: "gini", "entropy".
        max_depth : Optional[int]
            Ограничение глубины дерева. Если None - глубина не ограничена.
        min_samples_leaf : int
            Минимальное количество элементов в каждом листе дерева.

        """
        self.root = None
        if criterion == "gini":
            self.criterion = gini
        else:
            self.criterion = entropy
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = int(np.sqrt(X.shape[1])) if max_features == "auto" else max_features
        self.bag, self.oob = bagging(X, y)

    def fit(self) -> NoReturn:
        """
        Строит дерево решений по обучающей выборке.

        Parameters
        ----------
        X : np.ndarray
            Обучающая выборка.
        y : np.ndarray
            Вектор меток классов.
        """
        X, y = self.bag
        self.root = self.build(X, y, depth=0)

    def build(self, X, y, depth):

        max_gain = -float("inf")
        best_feature = None
        f = np.random.choice(X.shape[1], size=self.max_features, replace=False)
        for feature in f:
            right_mask = X[:, feature] == 1
            left_mask = X[:, feature] == 0
            cur_gain = gain(y[left_mask], y[right_mask], self.criterion)

            if max_gain < cur_gain:
                max_gain, best_feature = cur_gain, feature

        right_mask = X[:, best_feature] == 1
        left_mask = X[:, best_feature] == 0

        if depth == self.max_depth or np.sum(left_mask)<= self.min_samples_leaf or np.sum(right_mask) <= self.min_samples_leaf:
            return DecisionTreeLeaf(y)

        left = self.build(X[left_mask], y[left_mask], depth=depth+1)
        right = self.build(X[right_mask], y[right_mask], depth=depth+1)

        return DecisionTreeNode(split_dim=best_feature, split_value=1,
                 left=left,
                 right=right)


    def predict_proba(self, X: np.ndarray) ->  List[Dict[Any, float]]:
        """
        Предсказывает вероятность классов для элементов из X.

        Parameters
        ----------
        X : np.ndarray
            Элементы для предсказания.

        Return
        ------
        List[Dict[Any, float]]
            Для каждого элемента из X возвращает словарь
            {метка класса -> вероятность класса}.
        """
        res = []
        for x in X:
            node = self.root
            while not isinstance(node, DecisionTreeLeaf):
                if x[node.split_dim] < node.split_value:

                    node = node.left
                else:
                    node = node.right
            res.append(node.prob)
        return res

    def predict(self, X : np.ndarray) -> list:
        """
        Предсказывает классы для элементов X.

        Parameters
        ----------
        X : np.ndarray
            Элементы для предсказания.

        Return
        ------
        list
            Вектор предсказанных меток для элементов X.
        """
        proba = self.predict_proba(X)
        return [max(p.keys(), key=lambda k: p[k]) for p in proba]

In [7]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3,
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
tr = DecisionTreeClassifier(X, y)
tr.fit()

tr.predict(X[:20])

[0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1]

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [8]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=5, min_samples_leaf=5, max_features="auto", n_estimators=10):
        #self.criterion = gini if criterion == "gini" else entropy
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_estimators = n_estimators
        self.trees = []

    def fit(self, X, y):
        for i in range(self.n_estimators):
            tree = DecisionTreeClassifier(X, y, criterion=self.criterion, max_depth=self.max_depth,
                                          min_samples_leaf=self.min_samples_leaf, max_features=self.max_features)
            tree.fit()
            self.trees.append(tree)

    def predict(self, X):
        self.n_features = X.shape[1]
        pred = np.zeros((X.shape[0], self.n_estimators))
        for i, clf in enumerate(self.trees):
            pred[:, i] = clf.predict(X)
        pred = pred.astype(int)
        out = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            out[i] = np.bincount(pred[i]).argmax()
        return out

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [62]:
def feature_importance(rfc):
    out = np.zeros((rfc.n_estimators, rfc.n_features))
    for i, clf in enumerate(rfc.trees):
        X = clf.oob[0]
        y_true = clf.oob[1]
        y_pred = clf.predict(X)
        acc = accuracy_score(y_true, y_pred)
        for j in range(X.shape[1]):
            shuf = np.random.permutation(range(X.shape[0]))
            X_new = X.copy()
            X_new[:, j] = X[shuf, j]
            y_pred_j = clf.predict(X_new)
            out[i, j]= abs(accuracy_score(y_true, y_pred_j) - acc)

    return np.mean(out, axis=0)


def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [63]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [0.00568852 0.00678685 0.16604101 0.15839098 0.30886129 0.00360354]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [64]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [65]:
X, y_age, y_sex, features = read_dataset("vk.csv")
y_age = preprocessing.LabelEncoder().fit_transform(y_age)
y_sex = preprocessing.LabelEncoder().fit_transform(y_sex)
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [66]:
rfc = RandomForestClassifier(n_estimators=10)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.575031525851198
Most important features:
1. mudakoff
2. styd.pozor
3. 4ch
4. rhymes
5. pixel_stickers
6. reflexia_our_feelings
7. pravdashowtop
8. memeboizz
9. dayvinchik
10. ovsyanochan
11. ohhluul
12. i_d_t
13. ebashproklatie
14. thesmolny
15. webmland
16. nenormalnoo
17. pustota_diary
18. cook_bon
19. iloveu01
20. webestano


#### Пол

In [16]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.755359394703657
Most important features:
1. igm
2. 9o_6o_9o
3. 4ch
4. mudakoff
5. academyofman
6. cook_good
7. zerofat
8. woman.blog
9. 40kg
10. be.beauty
11. modnailru
12. i_d_t
13. rapnewrap
14. sh.cook
15. vinevinevine
16. reflexia_our_feelings
17. staaywithmee
18. be.women
19. beauty
20. great.food


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [56]:
rfc = CatBoostClassifier(
    logging_level='Silent',
    custom_loss=['Accuracy'],
    loss_function='MultiClass',
    use_best_model=True
)

In [57]:
X, y = synthetic_dataset(1000)
print("Accuracy:", np.mean(None == y))
print("Importance:", None)

Accuracy: 0.0
Importance: None


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [58]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

In [59]:
rfc = CatBoostClassifier(
    logging_level='Silent',
    loss_function='MultiClass'
)

#### Возраст

In [60]:
rfc.fit(X_train, y_age_train, eval_set=(X_eval, y_age_eval))
print("Accuracy:", np.mean(rfc.predict(X_test).flatten() == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(rfc.feature_importances_, features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.7528373266078184
Most important features:
1. ovsyanochan
2. styd.pozor
3. 4ch
4. mudakoff
5. rhymes
6. dayvinchik
7. leprum
8. rapnewrap
9. fuck_humor
10. xfilm
Accuracy: 0.7679697351828499
Most important features:
1. ovsyanochan
2. 4ch
3. styd.pozor
4. mudakoff
5. dayvinchik
6. rhymes
7. leprum
8. tumblr_vacuum
9. bot_maxim
10. rapnewrap


#### Пол

In [55]:
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test).flatten() == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(rfc.feature_importances_, features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.8764186633039092
Most important features:
1. 40kg
2. mudakoff
3. girlmeme
4. modnailru
5. femalemem
6. thesmolny
7. be.beauty
8. zerofat
9. 9o_6o_9o
10. reflexia_our_feelings
